#  Domain Gap Mitigation Strategy

### Visualizations for the manuscript
"Towards fully automated Inner Ear Analysis: Deep-Learning-based Joint Segmentation and Landmark Detection Framework" by Jannik Stebani, martin Blaimer, Simon Zabler, Tilmann Neun, Daniel M. Pelt and Kristen Rak

Code preface - collapse cells.

In [1]:
import sys
import json
import dataclasses
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

from pathlib import Path
from typing import Iterable, Optional, List
from copy import deepcopy
from collections import defaultdict

import plotly
import plotly.graph_objects as go

import h5py

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('C:/Users/Jannik/Desktop/viztools-rewrite/')

In [4]:
from nettools.pathutils import PathPair, parse_filepath, FilenameParsingError
from nettools.loader import Result, peek_dataset, load_autodispatch
import nettools.evaluate as nv
from nettools.slicedisplay import LabeledSliceDisplay, SliceDisplay
from nettools.vectortools import collect_dataset_landmarks
from nettools.vector import build_vectors, Vector, as_normalized_origin_vector, angle
from nettools.vectorplots import TraceBuilder

In [5]:
directory_clin = Path('G:/Cochlea/Manual_Segmentations/landmarked_transduced/clinical/')
directory_exvo = Path('G:/Cochlea/Manual_Segmentations/landmarked_transduced/exvivo/train/')
directory_wim = Path('C:/Users/Jannik/Desktop/wim-transduce/resampled-CT/')
directory_sib = Path('G:/Cochlea/dataset_sieber/transduced/')

In [6]:
clin_dataset_landmarks = collect_dataset_landmarks(directory_clin)
exvo_dataset_landmarks = collect_dataset_landmarks(directory_exvo)
wim_dataset_landmarks = collect_dataset_landmarks(directory_wim)
sib_dataset_landmarks = collect_dataset_landmarks(directory_sib)

C:\Users/Jannik/Desktop/viztools-rewrite\nettools\vectortools.py:111: UserWarning: Could not parse item: "G:\Cochlea\dataset_sieber\transduced\exportinfo.json" :: item skipped
  warnings.warn(


In [12]:
from collections import defaultdict

In [76]:
def transform_landmarks_datatypes(landmarks: dict[str, list[dict]]) -> dict[str, list[dict]]:
    """
    Transform coordinate arrays and numpy datatypes inside of
    landmark specifications  enable JSON serializability.
    """
    transformed = defaultdict(list)
    for ID, instance_landmarks in landmarks.items():
        transformed_instance_landmarks = []
        for landmark in instance_landmarks:
            transformed_landmark = {}
            for key, value in landmark.items():
                if isinstance(value, np.ndarray):
                    value = value.tolist()
                if isinstance(value, np.int32):
                    value = int(value)
                transformed_landmark[key] = value
            transformed_instance_landmarks.append(transformed_landmark)
        transformed[ID].append(transformed_instance_landmarks)
    return transformed
            

In [72]:
cd = transform_landmarks_arrays(clin_dataset_landmarks)

In [73]:
cld = deepcopy(clin_dataset_landmarks)

In [74]:
def export(data: dict, directory: Path, name: str) -> None:
    if not name.endswith('.json'):
        name = '.'.join((name, 'json'))
    path = directory / name
    # do not overwrite
    with path.open(mode='w') as outfile:
        json.dump(data, outfile, indent=4)

In [75]:
directory = Path('C:/Users/Jannik/Desktop/cochlea-spatial-investigation/data/')

In [78]:
landmarks_cache = {
    'spatialdata-B.json' : clin_dataset_landmarks,
    'spatialdata-A.json' : exvo_dataset_landmarks,
    'spatialdata-W.json' : wim_dataset_landmarks,
    'spatialdata-O.json' : sib_dataset_landmarks
}

for name, landmarkdata in landmarks_cache.items():
    landmarkdata = transform_landmarks_datatypes(landmarkdata)
    export(landmarkdata, directory, name)

In [ ]:
vectors_clin = build_vectors(clin_dataset_landmarks, 'RoundWindow', 'CochleaTop')
vectors_exvo = build_vectors(exvo_dataset_landmarks, 'RoundWindow', 'CochleaTop')
vectors_wim = build_vectors(wim_dataset_landmarks, 'RoundWindow', 'CochleaTop')
vectors_sib = build_vectors(sib_dataset_landmarks, 'RoundWindow', 'CochleaTop')

# Domain Gap Mitigation Strategy

## Spatial Orientation Analysis

For our domain gap investigation and subsequent mitigation strategy, we used the test-time augmentation framework to gain insights about the performance dropoff observed for some instances of the open source datasets. Using a wide array of test time augmentations, we observed a performance increase for certain rotation states of the input datasets.

To quantify the rotation state of a certain dataset instance, we compute the vector pointing from the round window landmark annotation towards the apex of the cochlea (internally called 'CochleaTop').
An example illustrative rendering for this vector is given below. We utilize the $\mathrm{IJK}$ voxel coordinate system for this task. 

In [ ]:
e_K = Vector(dataset_ID='KAXIS', base_landmark_label='ORIGIN', terminal_landmark_label='KDIR',
            base=np.array((0,0,0)), terminal=np.array((0,0,1)), shape=np.array((100, 100, 100)))

#### Angles for K-axis

We can calculate this parameter and take a look at the angles, that the RoundWindow->Apex vectors enclose with the $\mathrm{K}$-axis of the voxel cooridnate system.

In [ ]:
exvo_angles = [angle(e_K, v) for v in vectors_exvo]
clin_angles = [angle(e_K, v) for v in vectors_clin]
wim_angles = [angle(e_K, v) for v in vectors_wim]
sib_angles = [angle(e_K, v) for v in vectors_sib]

We observe that the instances of Datasets A and B all lie within a defined band of angles. Instances from dataset W and O have no such angle distribution. 

In [ ]:
fig, ax = plt.subplots()
ax.plot(exvo_angles, marker='x', ls='', label='A (exvivo)', color='tab:blue')
ax.plot(clin_angles, marker='x', ls='', label='B (clinical)', color='tab:green')

ax.plot(wim_angles, marker='x', ls='', label='W (Wimmer et al.)', color='tab:red')
ax.plot(sib_angles, marker='x', ls='', label='O (Sieber et al.)', color='tab:orange')

ax.set(xlabel='Dataset Index', ylabel='Angle $\mathrm{K}$ axis')
ax.set_title('Angle distribution of entire dataset corpus')
ax.legend()

We can further normalize of the vectors by scaling the components with the $L_2$ norm of the vector. This is performed in the cell below. The next step in the analysis is the computation of a mean orientation vector that indicates any preferential directions present in the datasets.

In [ ]:
nvectors_clin = [as_normalized_origin_vector(v) for v in vectors_clin]
nvectors_exvo = [as_normalized_origin_vector(v) for v in vectors_exvo]
nvectors_wim = [as_normalized_origin_vector(v) for v in vectors_wim]
nvectors_sib = [as_normalized_origin_vector(v) for v in vectors_sib]

In [ ]:
nvzip = zip(
    ['exvivo', 'clinical', 'wimmer', 'sieber'],
    [nvectors_exvo, nvectors_clin, nvectors_wim, nvectors_sib]
)
mean_str = ''
for name, vectorset in nvzip:
    mean = np.zeros(3)
    for v in vectorset:
        mean += v.delta
    mean /= len(vectorset)
    mean_str += f'v_{name}_mean = {mean}\n'
print(mean_str)

The values given above show that datasets A and B possess a large component along the second $\mathrm{J}$ axis pointing in negative direction.
This is in accordance with standard operating procedure in clinical practice.
 - The dataset W by Wimmer et al. has its largest component along the first $\mathrm{I}$ axis.
 - The dataset O by Sieber et al. has its largest component along the trailing $\mathrm{K}$ axis.

#### 3D Visualization

Having observed this, we proceed to 3D visualizations of the phenomenon.

In [ ]:
tb = TraceBuilder()
tb.tip_fraction = 0.9
tb.start_fraction = 0.98

Normalized vector traces

In [ ]:
tb.line_color = 'blue'
vectortraces_clin = tb.from_vectors(nvectors_clin)
tb.line_color = 'green'
vectortraces_exvo = tb.from_vectors(nvectors_exvo)
tb.line_color = 'red'
vectortraces_wim = tb.from_vectors(nvectors_wim)
tb.line_color = 'yellow'
vectortraces_sib = tb.from_vectors(nvectors_sib)

Raw vector traces

In [ ]:
vectors_sib

In [ ]:
tb.line_color = 'blue'
raw_vectortraces_clin = tb.from_vectors(vectors_clin)
tb.line_color = 'green'
raw_vectortraces_exvo = tb.from_vectors(vectors_exvo)
tb.line_color = 'red'
raw_vectortraces_wim = tb.from_vectors(vectors_wim)
tb.line_color = 'yellow'
raw_vectortraces_sib = tb.from_vectors(vectors_sib)

In [ ]:
global_layout_kwargs = {
    'legend_title_text' : 'Dataset Instance ID',
    'scene' :  dict(xaxis_title='I axis', yaxis_title='J axis', zaxis_title='K axis')
}

In [ ]:
fig = go.Figure()

fig.add_traces(raw_vectortraces_clin)
fig.add_traces(raw_vectortraces_exvo)
fig.update_layout(
    title='Raw IJK space Vector Plot: In-House datasets',
    **global_layout_kwargs
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_traces(vectortraces_clin)
fig.add_traces(vectortraces_exvo)
fig.update_layout(
    title='Normalized Origin Vector Plot: In-House Datasets',
    **global_layout_kwargs
)

fig.show()

We observe that the trainining and validation data instances (Dataset **A**, $N_{\mathrm{train}}=44$ and $N_{\mathrm{validation}}=5$, exvivo scans of cadaveric specimen) in blue and the test data instances in blue (Dataset **B**, $N_{\mathrm{test}}=10$) possess a preferential orientation in $\mathrm{IJK}$ voxel space.
This precipitates in the mean orientation vector that we calculated for both the datasets:
$$\langle \mathbf{v}_{\mathrm{A}}\rangle = (0.07166, -0.94792,  0.00671)^{\mathrm{T}}$$
$$\langle \mathbf{v}_{\mathrm{B}}\rangle = (-0.03682, -0.90414,  0.23284)^{\mathrm{T}}$$
The large component along the $\mathrm{J}$ axis encodes this observation quantitatively.
The consistent orientation of the inner ear anatomy in the datasets is in accordance with standard operatain procedure in clinical practice, where patients are placed in the scanner apparatus under the guidance of a domain expert.
Note that two dataset instances from Dataset **A** (instance ID 17 and 18) have differing angle orientation that falls outside of the described cone. 

We can further compute the angle between the vectors of instance '10_AV' and instance '04_BB' to get an estimate for the opening angle of the cone

In [ ]:
AV10 = list(filter(lambda v: v.dataset_ID == '10_AV', vectors_clin))[0]
BB04 = list(filter(lambda v: v.dataset_ID == '04_BB', vectors_clin))[0]
                                                                    
(AV10, BB04)

In [ ]:
angle(AV10, BB04)

We observe that the opening angle of the cone defined by the standard operating procedure in clinical practice is approximately $\theta_{o}\approx 60 ^{\circ}$

In [ ]:
fig = go.Figure()

fig.add_traces(raw_vectortraces_wim)
fig.add_traces(raw_vectortraces_sib)

fig.update_layout(
    title='Raw IJK Space Vector Plot: Open Source Datasets',
    **global_layout_kwargs
)

fig.show()

In [79]:
fig = go.Figure()

fig.add_traces(vectortraces_wim)
fig.add_traces(vectortraces_sib)

fig.update_layout(
    title='Normalized Origin Vector Plot: Open Source Datasets',
    **global_layout_kwargs
)

fig.show()

NameError: name 'vectortraces_wim' is not defined